In [4]:
import json
import random
from random import choice

In [5]:
def getcandidate(filepath):
    entity = []
    with open(filepath) as f:
        diclist = json.load(f)['entries']
        for dic in diclist:
            dic = list(dic.values())[0]
            for ops in dic["modifiedtripleset"]:
                entity.append(' '.join(ops['subject'].split('_')).replace('"', ''))
                entity.append(' '.join(ops['object'].split('_')).replace('"', ''))
    return entity

In [182]:
def change(wd):
    if random.random() < 0.5:
        return choice(allentity), True
    else:
        return wd, False

In [195]:
def makenoisy(gold, datadiclist):
    noisytext = gold
    poslist = [0 for i in range(len(gold.split(' ')))]
    # sum = 0
    entitys = []
    for datadic in datadiclist:
        entitys.append(' '.join(datadic['subject'].lower().split('_')))
        entitys.append(' '.join(datadic['object'].lower().split('_')))
        # print(datadic['subject'].lower())
        # print(noisytext)
    for entity in entitys:
        lis = noisytext.split(entity)
        length = len(lis)
        tmp = lis[0]
        posleft = []
        posout = []
        id = 0
        for wd in lis:
            if wd == '':
                lens = 0
            else:
                lens = len(wd.split(' '))
            posleft.append(poslist[id: id+len(lens)])
            posout.append(poslist[id+len(lens): id+len(lens)+len(entity.split(' '))])
            id += len(lens) + len(entity.split(' '))

        postmp = posleft[0]
        if length > 1:
            # sum += 1
            for i, (wd, posb, poso) in enumerate(zip(lis, posleft, posout)):
                if i > 0:
                    ch, wh = change(entity)
                    tmp += ch + wd
                    if wh:
                        postmp += [1 for mm in range(len(ch.split(' ')))] + posb
                    else:
                        postmp += poso + posb
            noisytext = tmp
            poslist = postmp


    return noisytext, poslist

In [196]:
def makepseudo(filepath):
    with open(filepath) as f:
        fsrcdata = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.src.data', 'w')
        fsrctext = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.src.text', 'w')
        ftgttext = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.text', 'w')
        ftgtpos = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.pos', 'w')
        ftgtedit = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.edit', 'w')

        diclist = json.load(f)['entries']
        for dic in diclist:
            dic = list(dic.values())[0]
            dic.pop('originaltriplesets')
            textlist = dic.pop('lexicalisations')
            textlist = [x['lex'] for x in textlist]
            datastr = '<category> '+dic['category']+' <shape> '+dic['shape']+' <type> '+dic['shape_type']+' <sep>'
            for datadic in dic["modifiedtripleset"]:
                datastr += ' <s> '+' '.join(datadic['subject'].split('_')).lower()+' <p> '+datadic['property']+' <o> '+' '.join(datadic['object'].split('_')).lower()
            for text in textlist:
                text = text.lower()
                ftgttext.write(text+'\n')
                noisytext, poslist = makenoisy(text, dic["modifiedtripleset"])
                fsrctext.write(noisytext + '\n')
                fsrcdata.write(datastr + '\n')
                ftgtpos.write(str(poslist) + '\n')
        ftgttext.close() 
        fsrcdata.close()
        fsrctext.close() 

In [6]:
allentity = []

In [7]:
allentity=getcandidate('webnlg_release_v2.1_dev.json')
# getcandidate('webnlg_release_v2.1_test.json')
# getcandidate('webnlg_release_v2.1_train.json')


In [198]:
makepseudo('webnlg_release_v2.1_dev.json')

In [8]:
len(allentity)

9412

In [11]:
allentity[3:9]

['2702.0',
 'Adirondack Regional Airport',
 '507',
 'Adirondack Regional Airport',
 'Harrietstown, New York',
 'Adolfo Suárez Madrid–Barajas Airport']